# Web scraping to create a dataset

In [97]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

from tqdm import tqdm

### Скрапинг адресов

In [101]:
st_accept = "text/html" # говорим веб-серверу, 
                        # что хотим получить html
# имитируем подключение через браузер Mozilla на macOS
st_useragent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.4 Safari/605.1.15"
# формируем хеш заголовков
headers = {
   "Accept": st_accept,
   "User-Agent": st_useragent,
}


def scraping_url(url: str, main_url:str, start_text: str) -> dict:

    result_lst = []

    # отправляем запрос с заголовками по нужному адресу
    src = requests.get(url, headers).text

    # инициализируем html-код страницы
    soup = BeautifulSoup(src, 'lxml')

    ul = soup.find("ul", {"class": "col3"})
    
    li = ul.find_all('li')

    for part in tqdm(li):

        # Находим только нужные улицы и переулки и тд.
        if 'class' in part.attrs:
            continue

        # Обрабатываем только нужные дома.
        else:
            lst = part.get_text().split()

            # индекс места.
            index = lst[0]
            # Название места: Александровский бульвар, 1-й Затонский пер. и тд
            place_name = ' '.join(lst[1:])
            # Ссылка с домами по месту.
            url = part.a['href']
            # print(f'{index = }\n{place_name = }\n{url = }')


            # Переходим по новой ссылке с домами.
            new_url = main_url + url
            new_src = requests.get(new_url, headers).text
            new_soup = BeautifulSoup(new_src, 'lxml')

            # Получаем дома.
            td = new_soup.find_all('td')
            houses = td[-1].get_text().split(', ')

            # Добавляем новый адрес в список.
            for house in houses:

                if house != 'Все':

                    result = f'{start_text}, {place_name}, {house}, индекс {index}'
                    
                    result_lst.append(result)

    return result_lst

In [99]:
MAIN_URL = 'https://index.kodifikant.ru'
CITY_URL = '/ru/2400000100000'

START_TEXT = 'Российская Федерация, Красноярский край, г. Красноярск'

In [102]:
result = scraping_url(f'{MAIN_URL}{CITY_URL}', MAIN_URL, START_TEXT)

100%|██████████████████████████████████████████████| 1341/1341 [16:44<00:00,  1.34it/s]


### Создаём dataframe и сохраним

In [111]:
df = pd.DataFrame({'target': result})

df.to_csv('test.csv', index=False)